### Load the model and make prediction
##### Input Args:

doc_dir = directory where the file with comments to be predicted

doc2vec_model_name = doc2vec model already trained

trained_clf = Trained classifier


In [26]:
from gensim.models import Doc2Vec
import re
import os
import random
from sklearn import metrics
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [27]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [28]:
def extract_words(sent):
    sent = sent.lower()
    sent = re.sub(r'<[^>]+>', ' ', sent) #strip html tags
    sent = re.sub(r'(\w)\'(\w)', '\1\2', sent) # remoce apostrophes
    sent = re.sub(r'\W',' ', sent) # remove punctuation
    sent = re.sub(r'\s+', ' ', sent) # remove repeated space
    sent = sent.strip()
    return sent.split()

In [29]:
#Load doc to vec model
doc2vec_model_name = 'reviews.d2v'
doc2vec_model = Doc2Vec.load(doc2vec_model_name)

In [30]:
#Test the trained doc2vec model
cosine_similarity(
[doc2vec_model.infer_vector(extract_words("This is very bad video. I don't like it"))],
[doc2vec_model.infer_vector(extract_words("video sucks."))])

array([[0.72811055]], dtype=float32)

In [31]:
#Converts the comments to vector using doc2vec model trianed earlier
def get_doc2vec(model, doc_dir,comment_col_name='Comment'):
    comments_dfs = []
    
    for fname in sorted(os.listdir(doc_dir)):
        fname_full = doc_dir + "/" + fname
        print("files being read : {0}".format(fname_full))
        data = pd.read_csv(fname_full, sep='\t', header='infer')
        data['FNAME'] = fname
        data['WORDS'] = data[comment_col_name].map(lambda x: extract_words(x))
        data['SENTVECS'] = data['WORDS'].map(lambda x: model.infer_vector(x, steps=10))
        comments_dfs.append(data)

    comments = pd.concat(comments_dfs)
    return comments

In [32]:
doc_dir = "../indata"
comments = get_doc2vec(doc2vec_model, doc_dir)

files being read : ../indata/EdSheeran.csv
files being read : ../indata/Maroon5.csv
files being read : ../indata/SuperBowlLI.csv
files being read : ../indata/bing-bang-theory.csv
files being read : ../indata/bitcoin-vs-ethereum.csv
files being read : ../indata/coldpay.csv
files being read : ../indata/eminem-not-affraid.csv
files being read : ../indata/federer-rafael.csv
files being read : ../indata/formula1-2018.csv
files being read : ../indata/got8.csv
files being read : ../indata/house-of-cards.csv
files being read : ../indata/iphone-xs-vs-note9.csv
files being read : ../indata/mac-vs-windows.csv
files being read : ../indata/nfl-highlights.csv
files being read : ../indata/oculus.csv
files being read : ../indata/samsung-vr.csv
files being read : ../indata/tonight-show.csv
files being read : ../indata/tswift.csv
files being read : ../indata/warriors.csv


In [33]:
comments.head(5)

,Comment,CreateTimeStamp,Type,videoID,videoTitle,FNAME,WORDS,SENTVECS
0,December everyone,2018-12-06T06:05:20.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[december, everyone]","[-0.06741282, -0.040171873, 0.02855836, 0.0490..."
1,the sweetest song ever in love with this song,2018-12-06T05:58:13.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[the, sweetest, song, ever, in, love, with, th...","[0.14719874, 0.3780378, 0.39704219, -0.2337952..."
2,Love this song,2018-12-06T05:58:04.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[love, this, song]","[0.17206766, 0.34257627, 0.1286713, -0.1261767..."
3,m gonna kill that son of bitch,2018-12-06T05:32:23.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[m, gonna, kill, that, son, of, bitch]","[0.122253135, -0.043197047, -0.057601914, 0.18..."
4,Wuuuuaaaaoooo,2018-12-06T05:31:54.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,[wuuuuaaaaoooo],"[-0.00095944904, -0.0033798655, -0.004097915, ..."


In [34]:
print("No of files : {0}".format(comments.FNAME.unique()))
print("No of Type : {}".format(comments.Type.unique()))

No of files : ['EdSheeran.csv' 'Maroon5.csv' 'SuperBowlLI.csv' 'bing-bang-theory.csv'
 'bitcoin-vs-ethereum.csv' 'coldpay.csv' 'eminem-not-affraid.csv'
 'federer-rafael.csv' 'formula1-2018.csv' 'got8.csv' 'house-of-cards.csv'
 'iphone-xs-vs-note9.csv' 'mac-vs-windows.csv' 'nfl-highlights.csv'
 'oculus.csv' 'samsung-vr.csv' 'tonight-show.csv' 'tswift.csv'
 'warriors.csv']
No of Type : ['Comment' 'Reply']


In [35]:
trained_clf = 'doc2vec_spam_rf.model'
loaded_clf = joblib.load(trained_clf)

In [36]:
comments['SPAM_IND_RF'] = loaded_clf.predict(comments['SENTVECS'].tolist())

In [37]:
comments.head(10)

,Comment,CreateTimeStamp,Type,videoID,videoTitle,FNAME,WORDS,SENTVECS,SPAM_IND_RF
0,December everyone,2018-12-06T06:05:20.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[december, everyone]","[-0.06741282, -0.040171873, 0.02855836, 0.0490...",0
1,the sweetest song ever in love with this song,2018-12-06T05:58:13.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[the, sweetest, song, ever, in, love, with, th...","[0.14719874, 0.3780378, 0.39704219, -0.2337952...",0
2,Love this song,2018-12-06T05:58:04.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[love, this, song]","[0.17206766, 0.34257627, 0.1286713, -0.1261767...",0
3,m gonna kill that son of bitch,2018-12-06T05:32:23.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[m, gonna, kill, that, son, of, bitch]","[0.122253135, -0.043197047, -0.057601914, 0.18...",0
4,Wuuuuaaaaoooo,2018-12-06T05:31:54.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,[wuuuuaaaaoooo],"[-0.00095944904, -0.0033798655, -0.004097915, ...",0
5,sa m sica es especial estaba en Killeen Texas,2018-12-06T05:31:40.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[sa, m, sica, es, especial, estaba, en, killee...","[0.46708402, -0.24910924, -0.21734653, 0.22903...",0
6,December,2018-12-06T05:31:13.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,[december],"[-0.03245747, -0.01236202, -0.050508488, -0.01...",0
7,Krlh cad os br,2018-12-06T05:29:04.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[krlh, cad, os, br]","[-0.13024846, -0.15833798, 0.071359836, 0.0634...",0
8,our song,2018-12-06T05:25:32.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[our, song]","[0.21713412, 0.16385709, 0.1639809, -0.0806052...",0
9,se parece el principe Harry poquito,2018-12-06T05:24:01.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[se, parece, el, principe, harry, poquito]","[-0.07974294, 0.10102428, 0.1363025, 0.3521796...",0


In [38]:
trained_clf = 'doc2vec_spam_kn.model'
loaded_clf = joblib.load(trained_clf)

In [39]:
comments['SPAM_IND_KN'] = loaded_clf.predict(comments['SENTVECS'].tolist())

In [40]:
comments.head(10)

,Comment,CreateTimeStamp,Type,videoID,videoTitle,FNAME,WORDS,SENTVECS,SPAM_IND_RF,SPAM_IND_KN
0,December everyone,2018-12-06T06:05:20.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[december, everyone]","[-0.06741282, -0.040171873, 0.02855836, 0.0490...",0,0
1,the sweetest song ever in love with this song,2018-12-06T05:58:13.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[the, sweetest, song, ever, in, love, with, th...","[0.14719874, 0.3780378, 0.39704219, -0.2337952...",0,0
2,Love this song,2018-12-06T05:58:04.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[love, this, song]","[0.17206766, 0.34257627, 0.1286713, -0.1261767...",0,0
3,m gonna kill that son of bitch,2018-12-06T05:32:23.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[m, gonna, kill, that, son, of, bitch]","[0.122253135, -0.043197047, -0.057601914, 0.18...",0,0
4,Wuuuuaaaaoooo,2018-12-06T05:31:54.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,[wuuuuaaaaoooo],"[-0.00095944904, -0.0033798655, -0.004097915, ...",0,0
5,sa m sica es especial estaba en Killeen Texas,2018-12-06T05:31:40.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[sa, m, sica, es, especial, estaba, en, killee...","[0.46708402, -0.24910924, -0.21734653, 0.22903...",0,1
6,December,2018-12-06T05:31:13.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,[december],"[-0.03245747, -0.01236202, -0.050508488, -0.01...",0,0
7,Krlh cad os br,2018-12-06T05:29:04.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[krlh, cad, os, br]","[-0.13024846, -0.15833798, 0.071359836, 0.0634...",0,0
8,our song,2018-12-06T05:25:32.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[our, song]","[0.21713412, 0.16385709, 0.1639809, -0.0806052...",0,0
9,se parece el principe Harry poquito,2018-12-06T05:24:01.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,"[se, parece, el, principe, harry, poquito]","[-0.07974294, 0.10102428, 0.1363025, 0.3521796...",0,0


In [41]:
out_df = comments[['Comment','CreateTimeStamp','Type', 'videoID', 'videoTitle','FNAME', 'SPAM_IND_RF','SPAM_IND_KN']]

In [42]:
out_df.head(5)

,Comment,CreateTimeStamp,Type,videoID,videoTitle,FNAME,SPAM_IND_RF,SPAM_IND_KN
0,December everyone,2018-12-06T06:05:20.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,0,0
1,the sweetest song ever in love with this song,2018-12-06T05:58:13.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,0,0
2,Love this song,2018-12-06T05:58:04.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,0,0
3,m gonna kill that son of bitch,2018-12-06T05:32:23.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,0,0
4,Wuuuuaaaaoooo,2018-12-06T05:31:54.000Z,Comment,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),EdSheeran.csv,0,0


In [43]:
out_file_name = '../outdata/video_comments_analysis_spam.csv'
out_df.to_csv(out_file_name, sep='\t', index=False)

In [44]:
print('****End of script****')

****End of script****
